<a href="https://colab.research.google.com/github/mmaronbr/alura-imersao-ia/blob/main/organizador_emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependências da Aplicação

Dependências da Generative AI

In [1]:
!pip install -U -q google.generativeai

Dependências do Gmail

In [2]:
!pip install -U -q google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 24.7 MB/s eta 0:00:00


# Importação das Bibliotecas

Bibliotecas utilizadas para acesso à API da Generative AI

In [3]:
import google.generativeai as genai

Bibliotecas utilizadas para acesso ao Gmail

In [7]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from google.colab import userdata

# Configurações

Definição da Chave da API da Generative AI

**IMPORTANTE!** Cadastre a secret `GOOGLE_API_KEY` na Seção de Secrets do Colab.

In [5]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Configurações para acesso ao Gmail

In [21]:
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

CLIENT_ID = userdata.get('CLIENT_ID')
CLIENT_SECRET = userdata.get('CLIENT_SECRET')
PROJECT_ID = userdata.get('PROJECT_ID')

JSON_TMP = f'''{{
    "installed": {{
      "client_id": "{CLIENT_ID}",
      "project_id": "{PROJECT_ID}",
      "auth_uri": "https://accounts.google.com/o/oauth2/auth",
      "token_uri": "https://oauth2.googleapis.com/token",
      "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
      "client_secret": "{CLIENT_SECRET}",
      "redirect_uris": ["http://localhost"]
    }}
  }}'''

# Cria o arquivo de template client_secret.json
with open("client_secret.json", "w") as client_secret:
  client_secret.write(JSON_TMP)


# Sandbox

Verifica se as configurações foram realizadas com sucesso para Generative AI

In [6]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content("Give me python code to sort a list")
print(response.text)

```python
# Example list
list1 = [5, 3, 1, 2, 4]

# Sort the list in ascending order
list1.sort()

# Print the sorted list
print(list1)  # Output: [1, 2, 3, 4, 5]

# Sort the list in descending order using the reverse parameter
list1.sort(reverse=True)

# Print the sorted list
print(list1)  # Output: [5, 4, 3, 2, 1]
```


In [13]:
import pathlib
pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

35

In [17]:
!gcloud auth application-default login \
  --no-browser --client-id-file /content/client_secret.json \
  --scopes https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning,https://www.googleapis.com/auth/generative-language.retriever


ERROR: gcloud crashed (JSONDecodeError): Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

If you would like to report this issue, please run the following command:
  gcloud feedback

To check gcloud for common problems, please run the following command:
  gcloud info --run-diagnostics
